In [1]:
# Replace 'YOUR_API_KEY' with your actual OpenAI API key
import os
os.environ['OPENAI_API_KEY'] = 'YOUR_API_KEY' #Sets your ‘OPENAI_API_KEY’ Environment Variable

In [ ]:
!pip install langchain

In [ ]:
!pip install openai

Using OpenAI LLM


In [5]:
from langchain.llms import OpenAI #Importing the OpenAI LLM

llm = OpenAI(temperature = 0.6)
# creates an instance of the OpenAI language model. The temperature parameter is being set to 0.6,
#which is a hyperparameter that controls the randomness of the model's output. Higher values (e.g., 1.0)
#make the output more random, while lower values (e.g., 0.2) make it more deterministic

name = llm("I want to make an Action movie. Suggest a catchy name for this.")
# OpenAI language model will use this prompt to generate a response

In [ ]:
print(name)

Using LLM Chain with PromptTemplate

In [ ]:
from langchain.prompts import PromptTemplate

prompt_template_name = PromptTemplate(
    input_variables = ['genre'], #prompt template expects a value for the 'genre' variable to be provided when creating a prompt.
    template = "I want to make a {genre} movie. Suggest a catchy name for this." #This is a string template for the prompt.
    #The {genre} placeholder will be replaced with the actual value when the template is used.
)

prompt_template_name.format(genre = 'Action') #this generates the following specific prompt: "I want to make an Action movie. Suggest a catchy name for this."

In [ ]:
from langchain.chains import LLMChain

chain = LLMChain(llm = llm, prompt = prompt_template_name)
chain.run("Action")

# This chain combines the llm's capabilities with the structured prompt template
# to generate a response based on the given input - "Action" movie.

Using Simple Sequential Chain with PromptTemplate

In [8]:
from langchain.llms import OpenAI #Importing the OpenAI LLM
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain


llm = OpenAI(temperature = 0.6)

prompt_template_name = PromptTemplate(
    input_variables = ['genre'],
    template = "I want to make a {genre} movie. Suggest a catchy name for this."
)

movie_name_chain = LLMChain(llm = llm, prompt = prompt_template_name)
# like before we are creating a chain to get the movie name

prompt_template_name = PromptTemplate(
    input_variables = ['movie_name'],
    template = "Suggest some actors for {movie_name}. Return it as a comma separated list."
)

movie_actors_chain = LLMChain(llm = llm, prompt = prompt_template_name)
# creating a chain to get the movie actors

In [ ]:
from langchain.chains import SimpleSequentialChain

chain = SimpleSequentialChain(chains = [movie_name_chain, movie_actors_chain])
response = chain.run("Action")

# using SimpleSequentialChain we are adding all the chains and getting the response.
# the output of movie_name_chain will be used as input to movie_actors_chain in a sequential manner.
print(response)

Using Sequential Chain with PromptTemplate

In [16]:
from langchain.llms import OpenAI #Importing the OpenAI LLM
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain


llm = OpenAI(temperature = 0.6)

prompt_template_name = PromptTemplate(
    input_variables = ['genre'],
    template = "I want to make a {genre} movie. Suggest a catchy name for this."
)

movie_name_chain = LLMChain(llm = llm, prompt = prompt_template_name, output_key="movie_name")

prompt_template_name = PromptTemplate(
    input_variables = ['movie_name'],
    template = "Suggest some actors for {movie_name}. Return it as a comma separated list."
)

movie_actors_chain = LLMChain(llm = llm, prompt = prompt_template_name, output_key = "movie_actors")

prompt_template_name = PromptTemplate(
    input_variables = ['movie_name'],
    template = "Write a small opening scene for this movie: {movie_name}."
)

movie_scene_chain = LLMChain(llm = llm, prompt = prompt_template_name, output_key = "movie_scene")
# this code sets up two chains (movie_name_chain and movie_actors_chain) that use the same language model (llm)
# but different prompt templates to first generate a catchy movie name based on a genre and then suggest actors
# for that movie based on the generated movie name. The output_key is used to associate the output of each chain
# with the appropriate variable, either "movie_name" or "movie_actors."

In [17]:
from langchain.chains import SequentialChain

chain = SequentialChain(
    chains = [movie_name_chain, movie_actors_chain, movie_scene_chain],
    input_variables = ['genre'],
    output_variables = ['movie_name', 'movie_actors', 'movie_scene']
)

movie_info = chain({'genre':'Action'})

# this code sets up a sequential chain of language model operations using the SequentialChain class.
# It combines two sub-chains (movie_name_chain and movie_actors_chain) into a sequence,
# specifies input and output variables for the entire chain, and then runs the chain with input data
# related to a movie's genre ('Action' in this case). The result, stored in movie_info, contains information about the movie's name and suggested actors.


In [ ]:
print(movie_info)

In [19]:
movie_name = movie_info['movie_name'].strip()
movie_actors_str = movie_info['movie_actors'].strip()
movie_actors = [actor.strip() for actor in movie_actors_str.split(',')]
movie_scene = movie_info['movie_scene'].strip()

In [ ]:
print(movie_name)

In [ ]:
print(movie_actors)

In [ ]:
print(movie_scene)

Searching Wikipedia

In [ ]:
!pip install wikipedia

In [26]:
from langchain.document_loaders import WikipediaLoader

In [27]:
docs = WikipediaLoader(query=movie_actors[0], load_max_docs=2).load()

# fetches information from Wikipedia about a specific movie actor.
# It queries Wikipedia with the actor's name (retrieved from movie_actors[0]) and limits the retrieval to a maximum of 2 documents.
# The retrieved data is then stored in the variable docs - docs[0] and docs[1]


In [ ]:
docs[0].page_content

In [ ]:
docs[1].page_content

Extracting Information from Wikipedia Documents

In [30]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import create_extraction_chain # using create_extraction_chain to extract specific information from text data.

# define a schema, which is a structured way to specify the properties or pieces of information you want to extract from text data.
schema = {
    "properties": {
        "name": {"type": "string"},
        "birth_date": {"type": "string"},
        "movie_names": {"type": "string"},
    },
}

In [31]:
# text data from where to extract
inp = docs[0].page_content

In [32]:
# Run chain
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo")
chain = create_extraction_chain(schema, llm)
result = chain.run(inp)

# Gives the result containing the extracted information structured according to the schema.

In [ ]:
print(result)

In [34]:
actor_name = result[0]['name'].strip()
actor_birth_date = result[0]['birth_date'].strip()
actor_movies = result[0]['movie_names'].strip()
actor_movies = [movie.strip() for movie in actor_movies.split(',')]

In [ ]:
print(actor_movies)

In [39]:
import openai

In [41]:
text = docs[0].page_content
Query = "Name the awards this person has received."

def answer_query(text, query):
  prompt = f"{text}. Using the above text as context, answer this query:{Query}"

  # Call the OpenAI API to answer our query
  response = openai.Completion.create(
          engine="text-davinci-003",
          prompt=prompt,
          max_tokens=256, # Set the maximum number of tokens for the summary, You can adjust this value based on your desired output length
          temperature=0.1 # can adjust this value in range [0-1]; controls the diversity of the generated responses.
          # Lower temperatures result in more predictable responses, while higher temperatures result in more variable responses.
      )

  # Extract and print the generated result
  result = response.choices[0].text.strip()
  return result

In [ ]:
print(answer_query(text, Query))

In [44]:
# Replace 'YOUR_API_KEY' with your actual OpenAI API key

openai.api_key = 'YOUR_API_KEY'

Generating a Movie Poster

In [ ]:
print(movie_scene)

In [51]:
def generate_image(movie_scene):
  prompt = f"A digital art that depicts the scene: {movie_scene}"
  response = openai.Image.create(
    prompt=prompt,
    n=1
  )
  image = response["data"][0]["url"]
  return image

# this code defines a function that generates a movie poster image based on a movie name using the OpenAI API.
# The function takes the movie name as input, formats a prompt, sends it to the API to generate the image, and returns the URL of the generated image in the 'image' variable.

image = generate_image(movie_scene)
# our case - The sun was beginning to set on the horizon, casting an orange glow over the deserted cityscape.
# In the center of the city, two figures stood facing each other, their silhouettes illuminated by the fading light.
# The tension between them was palpable, and the air was thick with anticipation. Suddenly, a loud bell rang out, signaling the start of the ultimate showdown.


In [ ]:
print(image)